In [6]:
from tinyec import registry
from Crypto.Cipher import AES
import hashlib, secrets, binascii


In [8]:
def encrypt_AES_GCM(msg, secretKey):
  aesCipher = AES.new(secretKey, AES.MODE_GCM)
  ciphertext, authTag = aesCipher.encrypt_and_digest(msg)
  return (ciphertext, aesCipher.nonce, authTag)
def decrypt_AES_GCM(ciphertext, nonce, authTag, secretKey):
  aesCipher = AES.new(secretKey, AES.MODE_GCM, nonce)
  plaintext = aesCipher.decrypt_and_verify(ciphertext, authTag)
  return plaintext
def ecc_point_to_256_bit_key(point):
  sha = hashlib.sha256(int.to_bytes(point.x, 32, 'big'))
  sha.update(int.to_bytes(point.y, 32, 'big'))
  return sha.digest()
curve = registry.get_curve('brainpoolP256r1')
def encrypt_ECC(msg, pubKey):
  ciphertextPrivKey = secrets.randbelow(curve.field.n)
  sharedECCKey = ciphertextPrivKey * pubKey
  secretKey = ecc_point_to_256_bit_key(sharedECCKey)
  ciphertext, nonce, authTag = encrypt_AES_GCM(msg, secretKey)
  ciphertextPubKey = ciphertextPrivKey * curve.g
  return (ciphertext, nonce, authTag, ciphertextPubKey)
def decrypt_ECC(encryptedMsg, privKey):
  (ciphertext, nonce, authTag, ciphertextPubKey) = encryptedMsg
  sharedECCKey = privKey * ciphertextPubKey
  secretKey = ecc_point_to_256_bit_key(sharedECCKey)
  plaintext = decrypt_AES_GCM(ciphertext, nonce, authTag, secretKey)
  return plaintext

In [9]:
from Crypto.PublicKey import RSA
from Crypto.Signature.pkcs1_15 import PKCS115_SigScheme
from Crypto.Hash import SHA256
import binascii


keyPair = RSA.generate(bits=1024)
pubKey = keyPair.publickey()
flag=0
while(flag==0):
  print("-----------------SECURE PAYMENT APP------------------")
  print("Type 1 for credit card")
  print("Type 2 for debit card")
  print("Type 3 for UPI payemnt")
  print("Type 4 for exit ")
  
  method=int(input())

  if (method==4):
    exit(0)
  else:
    if(method==1 or method==2):
      card=input("Please enter card number")
      cvv=input("please enter cvv")

msg = b'Message for RSA signing'
hash = SHA256.new(msg)
signer = PKCS115_SigScheme(keyPair)
signature = signer.sign(hash)
print("Signature:", binascii.hexlify(signature))

# Verify valid PKCS#1 v1.5 signature (RSAVP1)
msg = b'Message for RSA signing'
hash = SHA256.new(msg)
verifier = PKCS115_SigScheme(pubKey)
try:
    verifier.verify(hash, signature)
    print("Signature is valid.")
except:
    print("Signature is invalid.")



Signature: b'958f698c33cff733f985c0e8aa48867b488b511b3e2cf2297c1c90154570bdec05954bb010d1cb3c90c27918234767bb0298c961ddb6ab93a2e9621080f144c234e90956d299103b1f56a5686c0aec5925df2611f115e652e4f528d5a46b311371a756a3e06be7064c33061ff24a7bba0e558b731629ab9e4b57e06d078bf57f'
Signature is valid.
Signature is invalid.


In [14]:
import Crypto
from Crypto.PublicKey import RSA
from Crypto import Random
import base64
from Crypto.Cipher import PKCS1_OAEP

def rsakeys():  
     length=1024  
     privatekey = RSA.generate(length, Random.new().read)  
     publickey = privatekey.publickey()  
     return privatekey, publickey
def encrypt(rsa_publickey,plain_text):
     cipher_text=rsa_publickey.encrypt(plain_text,32)[0]
     b64cipher=base64.b64encode(cipher_text)
     return b64cipher
def decrypt(rsa_privatekey,b64cipher):
     decoded_ciphertext = base64.b64decode(b64cipher)
     plaintext = rsa_privatekey.decrypt(decoded_ciphertext)
     return plaintext
privatekey,publickey=rsakeys()
def sign(privatekey,data):
    return base64.b64encode(str((privatekey.sign(data,''))[0]).encode())
def verify(publickey,data,sign):
     return publickey.verify(data,(int(base64.b64decode(sign)),))

text=b"Hello Srikanth!"
ct=encrypt(publickey,text)
print(ct)
decrypt(privatekey,ct)

NotImplementedError: ignored

In [ ]:
import ast
from Crypto.PublicKey import RSA
from Crypto import Random
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.Signature.pkcs1_15 import PKCS115_SigScheme


random_generator = Random.new().read
key = RSA.generate(1024, random_generator)  

publickey = key.publickey()


encryptor = PKCS1_OAEP.new(publickey)
flag=0
while(flag==0):
  print("-----------------SECURE PAYMENT APP------------------")
  print("Type 1 for credit card")
  print("Type 2 for debit card")
  print("Type 3 for UPI payemnt")
  print("Type 4 for exit ")
  
  method=int(input())

  if (method==4):
    print("EXIT")
    exit(0)
    break
  elif (method==1 or method == 2):
    if(method==1 or method==2):
      
      card=input("Please enter card number")
      cvv=input("please enter cvv")
      value =card+cvv
      h=SHA256.new(value.encode())
      signature=pkcs1_15.new(key).sign(h)
      print(signature)
      value=value.encode()
      encrypted = encryptor.encrypt(value)

      print("encrypted data is =",encrypted)
      decryptor = PKCS1_OAEP.new(key)

      decrypted = decryptor.decrypt(ast.literal_eval(str(encrypted)))
      print("decrypted data = ",decrypted)
      try:
        pkcs1_15.new(publickey).verify(h, signature)
        print ("The signature is valid.")
        print("Transaction completed")
        print("----------------- THANK YOU ------------------")
      except (ValueError, TypeError):
        print ("The signature is not valid.")
        print("Transaction Failed please try again")
      
      

  else:
    upi=input("Enter UPI ID")
    print(upi)
    h=SHA256.new(upi.encode())
    signature=pkcs1_15.new(key).sign(h)
    print("Signature is = ",signature)
    encrypted = encryptor.encrypt(upi.encode())
    print("encrypted data is =",encrypted)
    decryptor = PKCS1_OAEP.new(key)

    decrypted = decryptor.decrypt(ast.literal_eval(str(encrypted)))
    print("decrypted data = ",decrypted)
    h=SHA256.new(decrypted)
    try:
      pkcs1_15.new(publickey).verify(h, signature)
      print ("The signature is valid.")
      print("Transaction completed")
      print("----------------- THANK YOU ------------------")
      
    except (ValueError, TypeError):
      print ("The signature is not valid.")
      print("Transaction Failed please try again")






-----------------SECURE PAYMENT APP------------------
Type 1 for credit card
Type 2 for debit card
Type 3 for UPI payemnt
Type 4 for exit 
3
Enter UPI IDhemendra
hemendra
Signature is =  b'ai\xd4\x99/gOV\xf5J\xcf\xc8y\n\xb0{\xe7\xe6\xc2\x03V\x1c\x99\r\xebZ\x84\xae\x8e2\xf1\xd0 \xc9\x94\x9d\x97+\x1a\x0b\xbe\x97\xea\xc2xlI\x91i*NT\xeb\x0e\x08~[\xe7\xc5\xdc\xec{\xc8\xf5x\xdd\xb6\xad\xf8C\x8c\xb26\x13\xca^\x96\x1f\xfa\x9f\xdd8\x85\x8c\xdc\x07!#yE\xcc\xe9\xf2P\xe5\t\xf9;\xb9*\x99)\x10\x8ci[\xe7\x1a\xe9?\xa3\x07\xd1\x12\x7fS\x8c\xc3\x1dk\x96\xca\xae~d\xe5\x14\x05'
encrypted data is = b'{\x87";\x8a\xa8\x87\xe6\xaf`\xb7\x02Q\x12\x13J\x96j\xea\xefw8\xf4\xcb\x05\xab\x8f/\x8a\xea\xaeo\xd2\xb2\xf7\x1d|W\xe85\x8f\xf8>)C\xbf[\xd0]\x14\x1e[z\x80\x84,\x18\xc8\x1d\xd4\x1c,\x17e\xd7\xde&\x89\xa4S\xb3l\x8e\x19\x86\xc6\xea\x05d\xba;-\xc9\xbdvv\x13\x7fb!T\xe0=\xef@\xe7.\x9d\xac-\x8ae|\x05f\x1c%\x86\xbdD/\xf2\x117B9.?\xef\x07\xa4N7ZG\x1b\xed\xb8'
decrypted data =  b'hemendra'
The signature is valid.
Transac